**Use the evaluation_results.csv file as input for this code to find topsis scroe and rank the models**

In [1]:
#!pip install scikit-criteria
import numpy as np
import pandas as pd
from sklearn import preprocessing
from skcriteria import mkdm

In [2]:
evaluation_results = pd.read_csv("/content/topsis_input.csv")
print(evaluation_results)

                                    Model  BLEU Score  Semantic Coherence  \
0                      google-t5/t5-large    0.352185            0.768574   
1          pszemraj/led-base-book-summary    0.314918            0.688963   
2  hardikJ11/bart-base-finetuned-cnn-news    0.334391            0.710904   
3        philschmid/bart-large-cnn-samsum    0.439666            0.632890   
4      BigSneed/autotrain-sima-2512277279    0.354885            0.771122   

   Factual Accuracy  Content Coverage  Readability  
0          0.447016          0.391597     3.856667  
1          0.544512          0.482278     3.578667  
2          0.390311          0.342793     3.858000  
3          0.539266          0.288917     5.119333  
4          0.428660          0.382961     3.981333  


**Import Dependencies**

**You may adjust the weights based on the importance of each criterion as per your choice**

Changing the weights may change the output

In [3]:
weights = [0.25, 0.25, 0.25, 0.25, 0.25]

**Normalize the scores and then calculate topsis scores and ranks for different models**

In [4]:
# Normalize the scores
normalized_scores = evaluation_results.iloc[:, 1:].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Convert the normalized scores to a numpy array
normalized_scores_array = normalized_scores.to_numpy()

# Define the ideal and negative-ideal solutions
ideal_solution = np.max(normalized_scores_array, axis=0)
negative_ideal_solution = np.min(normalized_scores_array, axis=0)

# Calculate the Euclidean distances to ideal and negative-ideal solutions
euclidean_distances_to_ideal = np.linalg.norm(normalized_scores_array - ideal_solution, axis=1)
euclidean_distances_to_negative_ideal = np.linalg.norm(normalized_scores_array - negative_ideal_solution, axis=1)

# Calculate the TOPSIS scores
topsis_scores = euclidean_distances_to_negative_ideal / (euclidean_distances_to_ideal + euclidean_distances_to_negative_ideal)

# Add TOPSIS scores to your evaluation_results DataFrame
evaluation_results["TOPSIS Score"] = topsis_scores

# Rank based on TOPSIS scores
evaluation_results["Rank"] = evaluation_results["TOPSIS Score"].rank(ascending=False)

# Display the final evaluation_results DataFrame
print(evaluation_results)

                                    Model  BLEU Score  Semantic Coherence  \
0                      google-t5/t5-large    0.352185            0.768574   
1          pszemraj/led-base-book-summary    0.314918            0.688963   
2  hardikJ11/bart-base-finetuned-cnn-news    0.334391            0.710904   
3        philschmid/bart-large-cnn-samsum    0.439666            0.632890   
4      BigSneed/autotrain-sima-2512277279    0.354885            0.771122   

   Factual Accuracy  Content Coverage  Readability  TOPSIS Score  Rank  
0          0.447016          0.391597     3.856667      0.478583   3.0  
1          0.544512          0.482278     3.578667      0.489552   2.0  
2          0.390311          0.342793     3.858000      0.276876   5.0  
3          0.539266          0.288917     5.119333      0.547647   1.0  
4          0.428660          0.382961     3.981333      0.472200   4.0  


**Saving the output csv file**

In [5]:
evaluation_results.to_csv("topsis_results.csv", index=False)